In [1]:
import json
from scipy import stats
import numpy as np
import pandas as pd

In [2]:
PATH_TO_JSON = '/cronus_data/avirinchipur/reasoning_for_psych/irt/gpt4_sr_fulldata_ranking.json'
combined_data_symp_rank = json.load(open(PATH_TO_JSON, 'rb'))

In [3]:
colnames = ['score_Anhedonia', 'score_Depressed_Mood', 'score_Insomnia_or_Hypersomnia', 'score_Fatigue', \
        'score_Poor_appetite_or_overeating', 'score_Worthlessness_or_Guilt', 'score_Difficulty_concentrating', \
        'score_Psychomotor_agitation_or_retardation', 'score_Suicidal_ideation']

In [4]:
combined_data_symp_rank['gpt4'].keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8'])

In [6]:
gpt4_symp_ranks_all = []
self_symp_ranks_all = []
for i in range(9):
    gpt4_symp_ranks_all.append(np.array(combined_data_symp_rank['gpt4'][str(i)]))
    self_symp_ranks_all.append(np.array(combined_data_symp_rank['self_report'][str(i)]))

gpt4_symp_ranks_all = np.array(gpt4_symp_ranks_all)
self_symp_ranks_all = np.array(self_symp_ranks_all)

print ("Shapes: ", gpt4_symp_ranks_all.shape, self_symp_ranks_all.shape)

spearman_corr = []
kendall_corr = []
for i in range(gpt4_symp_ranks_all.shape[1]):
    spearman_corr.append(stats.spearmanr(gpt4_symp_ranks_all[:,i], self_symp_ranks_all[:,i]))
    kendall_corr.append(stats.kendalltau(gpt4_symp_ranks_all[:,i], self_symp_ranks_all[:,i]))
    
# Report the 95% confidence interval of the Spearman correlation
spearman_corr = np.array(spearman_corr)[:, 0]
kendall_corr = np.array(kendall_corr)[:, 0]
print (f"95% confidence interval of Spearman correlation: {np.percentile(spearman_corr, 5).round(3)} - {np.percentile(spearman_corr, 95).round(3)}")
print (f"95% confidence interval of Kendall correlation: {np.percentile(kendall_corr, 5).round(3)} - {np.percentile(kendall_corr, 95).round(3)}")


Shapes:  (9, 500) (9, 500)
95% confidence interval of Spearman correlation: 0.883 - 0.983
95% confidence interval of Kendall correlation: 0.722 - 0.944


In [10]:
# Compute 95% confidence interval of the ranks of each symptom 
rank_bounds = {}
for i in range(9):
    gpt4_rank = gpt4_symp_ranks_all[i]
    self_rank = self_symp_ranks_all[i]
    gpt4_rank_stderr = (np.std(gpt4_rank) / np.sqrt(len(gpt4_rank))).round(2)
    self_rank_stderr = (np.std(self_rank) / np.sqrt(len(self_rank))).round(2)
    rank_bounds[colnames[i]] = [np.percentile(gpt4_rank, 2.5), np.percentile(gpt4_rank, 97.5), #gpt4_rank_stderr, 
                                np.percentile(self_rank, 2.5), np.percentile(self_rank, 97.5)#, self_rank_stderr
                                ]

pd.DataFrame(rank_bounds, index=['GPT4_lwr', 'GPT4_upr', 'Self_lwr', 'Self_upr']).T

,GPT4_lwr,GPT4_upr,Self_lwr,Self_upr
score_Anhedonia,7.0,9.0,7.0,7.0
score_Depressed_Mood,8.0,9.0,8.0,9.0
score_Insomnia_or_Hypersomnia,5.0,6.0,5.0,6.0
score_Fatigue,7.0,9.0,8.0,9.0
score_Poor_appetite_or_overeating,2.0,3.0,3.0,3.0
score_Worthlessness_or_Guilt,4.0,4.0,4.0,6.0
score_Difficulty_concentrating,5.0,6.0,4.0,6.0
score_Psychomotor_agitation_or_retardation,2.0,3.0,2.0,2.0
score_Suicidal_ideation,1.0,1.0,1.0,1.0


In [9]:
fulldata_ranks = pd.read_csv('/cronus_data/avirinchipur/reasoning_for_psych/irt/gpt4_sr_fulldata_ranking.csv')
fulldata_ranks


,gpt4,self_report,columns
0,7.0,7.0,score_Anhedonia
1,9.0,8.0,score_Depressed_Mood
2,6.0,6.0,score_Insomnia_or_Hypersomnia
3,8.0,9.0,score_Fatigue
4,2.0,3.0,score_Poor_appetite_or_overeating
5,4.0,5.0,score_Worthlessness_or_Guilt
6,5.0,4.0,score_Difficulty_concentrating
7,3.0,2.0,score_Psychomotor_agitation_or_retardation
8,1.0,1.0,score_Suicidal_ideation


In [17]:
p_vals = {}
for i in range(9):
    gpt4_symp_ranks = np.array(combined_data_symp_rank['gpt4'][str(i)])
    self_symp_ranks = np.array(combined_data_symp_rank['self_report'][str(i)])
    
    p_gpt4_lesser = 1 - (sum(gpt4_symp_ranks < self_symp_ranks)/len(gpt4_symp_ranks))
    p_gpt4_greater = 1 - (sum(gpt4_symp_ranks > self_symp_ranks)/len(gpt4_symp_ranks))
    p_equal = 1 - (sum(gpt4_symp_ranks == self_symp_ranks)/len(gpt4_symp_ranks))
    p_two_sided = 2*min(p_gpt4_lesser, p_gpt4_greater)
    
    p_vals[colnames[i]] = {'p_gpt4_lesser': p_gpt4_lesser, 'p_gpt4_greater': p_gpt4_greater, 'p_equal': p_equal, 'p_two_sided': p_two_sided}
    
pd.DataFrame(p_vals).T
    
    

,p_gpt4_lesser,p_gpt4_greater,p_equal,p_two_sided
score_Anhedonia,0.996,0.484,0.520,0.968
score_Depressed_Mood,0.990,0.172,0.838,0.344
score_Insomnia_or_Hypersomnia,0.950,0.808,0.242,1.616
score_Fatigue,0.108,0.998,0.894,0.216
score_Poor_appetite_or_overeating,0.094,1.000,0.906,0.188
score_Worthlessness_or_Guilt,0.440,0.996,0.564,0.880
score_Difficulty_concentrating,0.946,0.442,0.612,0.884
score_Psychomotor_agitation_or_retardation,1.000,0.094,0.906,0.188
score_Suicidal_ideation,1.000,1.000,0.000,2.000


In [19]:
# Apply mannwhitneyu
mannwhitney_results = {}
for i in range(1):
    gpt4_symp_ranks = np.array(combined_data_symp_rank['gpt4'][str(i)])
    self_symp_ranks = np.array(combined_data_symp_rank['self_report'][str(i)])
    _, p_two_sided = stats.mannwhitneyu(gpt4_symp_ranks, self_symp_ranks, alternative='two-sided', use_continuity=False, method='exact')
    _, p_less = stats.mannwhitneyu(gpt4_symp_ranks, self_symp_ranks, alternative='less', use_continuity=False, method='exact')
    _, p_greater = stats.mannwhitneyu(gpt4_symp_ranks, self_symp_ranks, alternative='greater', use_continuity=False, method='exact')
    mannwhitney_results[colnames[i]] = {'p_two_sided': p_two_sided, 'p_less': p_less, 'p_greater': p_greater}
    
pd.DataFrame(mannwhitney_results).T
    
    

KeyboardInterrupt: 